In [ ]:
import re
import pandas as pd
import itertools

In [ ]:
regex = re.compile("""([0-9:.]+) .* pt=([0-9]+), seqnum=([0-9]+), rtptime=([0-9]+)""")

In [ ]:
line = """0:30:44.095474760 1664553 0x5613835cc120 DEBUG        rtpjitterbuffer gstrtpjitterbuffer.c:2902:gst_rtp_jitter_buffer_chain:<rtpjitterbuffer0> 00m pt=97, seqnum=11642, rtptime=11642"""
regex.match(line)

In [ ]:
regex_identity = re.compile(""".*GstIdentity:([^: ]+): .* chain.* pts: ([0-9]+):([0-9]+):([0-9]+)\.([0-9]+).*""")
line = """/GstPipeline:pipeline0/GstIdentity:identity-from-rtspsrc: last-message = chain   ******* (identity-from-rtspsrc:sink) (15 bytes, dts: none, pts: 0:00:00.000000000, duration: none, offset: -1, offset_end: -1, flags: 00004040 discont tag-memory , meta: none) 0x55ea3d1f87e0"""
regex_identity.match(line)

In [ ]:
# Create log file with rtsp-camera-to-fakesink-rtptime.log.
# Requires gstrtpjitterbuffer.c with customized logging in gst_rtp_jitter_buffer_chain().
#   GST_DEBUG_OBJECT (jitterbuffer, "pt=%d, seqnum=%d, rtptime=%d", pt, seqnum, rtptime);

# logfile = '../../../tmp/rtptime.log'
logfile = '../../../tmp/rtsp-camera-to-fakesink-rtptime.log'

In [ ]:
%%time
data = []
data_identity = []
with open(logfile) as f:
    for line in itertools.islice(f, 0, int(1e10)):
        m = regex.match(line)
        if m:
            logtime = m.group(1)
            pt = int(m.group(2))
            seqnum = int(m.group(3))
            rtptime = int(m.group(4))            
            data += [(logtime, pt, seqnum, rtptime)]
        m = regex_identity.match(line)
        if m:
            element = m.group(1)
            hours = int(m.group(2))
            minutes = int(m.group(3))
            seconds = int(m.group(4))
            nanos = int(m.group(5))
            pts = ((hours * 60 + minutes) * 60 + seconds) * int(1e9) + nanos
            data_identity += [(element, pts, line)]

In [ ]:
len(data), len(data_identity)

In [ ]:
df = pd.DataFrame(data, columns=['logtime','pt','seqnum','rtptime'])

In [ ]:
df

In [ ]:
df.pt.value_counts()

In [ ]:
vdf = df[df.pt==96]

In [ ]:
vdf[['rtptime']].plot();

In [ ]:
pd.set_option('display.max_rows', 50)

In [ ]:
diff = vdf.rtptime.diff()
diff[diff != 0][:30]

In [ ]:
vdf.rtptime.diff().value_counts()

In [ ]:
(vdf.rtptime.diff() / 90000).value_counts()

In [ ]:
df_identity = pd.DataFrame(data_identity, columns=['element','pts','line'])

In [ ]:
df_identity

In [ ]:
df_identity['diff'] = df_identity[df_identity.element=="identity-from-rtspsrc"].pts.diff()

In [ ]:
df_identity

In [ ]:
(df_identity[df_identity.element=="identity-from-rtspsrc"].pts.diff() * 1e-9).value_counts()

In [ ]:
df_identity['diff'].value_counts()

In [ ]:
s = df_identity['line']

In [ ]:
s.str.contains('resync')

In [ ]:
df_identity.line.contains('resync')

In [ ]:
df_identity[df_identity.line.str.contains('resync')]

In [ ]:
df_identity[(df_identity['diff'] != 0) & (df_identity['diff'] != 50000000) & ~df_identity['diff'].isna()]

In [ ]:
pd.set_option('display.max_colwidth', None)
df_identity[df_identity['diff']]
#(df_identity['diff'] * 1e-9).value_counts()